<a href="https://colab.research.google.com/github/beniamine3155/Fine_Tuning_LLM_with_HuggingFace/blob/main/RLHF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reinforcement Learning with Human Feedback


In [1]:
!pip install trl transformers datasets peft accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 9.5 MB/s eta 0:00:00


## Supervised Fine Tuning (SFT)

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
import torch

In [2]:
data = [
    {"prompt": "What is AI?", "chosen": "AI means Artificial Intelligence.", "rejected": "AI is a kind of fruit."},
    {"prompt": "Explain Python.", "chosen": "Python is a programming language.", "rejected": "Python is a type of snake."},
    {"prompt": "Describe climate change.", "chosen": "Climate change refers to long-term shifts in temperatures and weather patterns.", "rejected": "Climate change is a myth."},
    {"prompt": "What causes rain?", "chosen": "Rain is caused by water vapor condensing and falling from clouds.", "rejected": "Rain is caused by people crying."},
    {"prompt": "Define photosynthesis.", "chosen": "Photosynthesis is the process by which plants convert sunlight into energy.", "rejected": "Photosynthesis is a type of dance."},
    {"prompt": "What is reinforcement learning?", "chosen": "Reinforcement learning is a type of machine learning where agents learn by interacting with an environment.", "rejected": "Reinforcement learning is about reinforcing weak WiFi signals."},
    {"prompt": "What is quantum computing?", "chosen": "Quantum computing uses quantum bits to perform complex computations more efficiently.", "rejected": "Quantum computing is about counting quanta of light."},
    {"prompt": "Who wrote Hamlet?", "chosen": "Hamlet was written by William Shakespeare.", "rejected": "Hamlet is a famous soccer player."},
    {"prompt": "Explain gravity.", "chosen": "Gravity is the force that attracts two bodies towards each other.", "rejected": "Gravity is a magic trick to make things fall."},
    {"prompt": "What is blockchain?", "chosen": "Blockchain is a decentralized digital ledger technology.", "rejected": "Blockchain is a chain made of blocks of stone."},
    {"prompt": "What is machine learning?", "chosen": "Machine learning is a subset of AI that enables systems to learn patterns from data.", "rejected": "Machine learning is when machines attend school."},
    {"prompt": "Define deep learning.", "chosen": "Deep learning is a subset of machine learning using neural networks with many layers.", "rejected": "Deep learning means studying deeply for long hours."},
    {"prompt": "What is data science?", "chosen": "Data science is the study of data to extract meaningful insights using statistics and algorithms.", "rejected": "Data science is the science of fixing computers."},
    {"prompt": "Explain the Internet.", "chosen": "The Internet is a global network connecting millions of computers for communication and information sharing.", "rejected": "The Internet is a big cloud floating in the sky."},
    {"prompt": "What is a database?", "chosen": "A database is an organized collection of structured information or data stored electronically.", "rejected": "A database is a base where people dance."},
    {"prompt": "Who was Albert Einstein?", "chosen": "Albert Einstein was a physicist known for the theory of relativity.", "rejected": "Einstein was a famous chef."},
    {"prompt": "What is an algorithm?", "chosen": "An algorithm is a step-by-step procedure to solve a problem or perform a task.", "rejected": "An algorithm is a type of musical rhythm."},
    {"prompt": "Define cloud computing.", "chosen": "Cloud computing provides computing resources over the Internet on demand.", "rejected": "Cloud computing means calculating the number of clouds in the sky."},
    {"prompt": "What is cybersecurity?", "chosen": "Cybersecurity protects systems, networks, and data from digital attacks.", "rejected": "Cybersecurity is about securing your bicycle online."},
    {"prompt": "Explain big data.", "chosen": "Big data refers to large and complex data sets that require special processing techniques.", "rejected": "Big data is a very tall computer."},
    {"prompt": "What is natural language processing?", "chosen": "NLP enables computers to understand and generate human language.", "rejected": "NLP is about speaking to plants naturally."},
    {"prompt": "What is a neural network?", "chosen": "A neural network is a computational model inspired by the human brain's structure.", "rejected": "A neural network is the internet inside your brain."},
    {"prompt": "Define ethics in AI.", "chosen": "AI ethics is the study of moral principles guiding AI development and use.", "rejected": "AI ethics means teaching robots to be polite."},
    {"prompt": "What is computer vision?", "chosen": "Computer vision allows computers to interpret and process visual information from the world.", "rejected": "Computer vision means wearing glasses while coding."},
    {"prompt": "Explain the purpose of an operating system.", "chosen": "An operating system manages hardware and software resources on a computer.", "rejected": "An operating system operates airplanes."},
    {"prompt": "What is the metaverse?", "chosen": "The metaverse is a virtual world where people interact through digital avatars.", "rejected": "The metaverse is a type of poem."},
    {"prompt": "What is cloud storage?", "chosen": "Cloud storage saves data on remote servers accessible via the Internet.", "rejected": "Cloud storage stores clouds for weather prediction."},
    {"prompt": "Explain the term ‘open source’.", "chosen": "Open source refers to software whose source code is freely available to modify and distribute.", "rejected": "Open source means software that stays open all day."},
    {"prompt": "Who was Isaac Newton?", "chosen": "Isaac Newton was a scientist known for his laws of motion and gravity.", "rejected": "Isaac Newton was an astronaut."},
    {"prompt": "What is an API?", "chosen": "An API is an interface that allows software applications to communicate with each other.", "rejected": "An API is a type of insect like an ant."},
    {"prompt": "Define cryptocurrency.", "chosen": "Cryptocurrency is a digital currency secured by cryptography.", "rejected": "Cryptocurrency is money you hide under a crypt."},
    {"prompt": "What is a chatbot?", "chosen": "A chatbot is an AI system designed to simulate conversation with humans.", "rejected": "A chatbot is a robot that makes tea."},
    {"prompt": "Explain Internet of Things (IoT).", "chosen": "IoT connects everyday devices to the Internet for data exchange and automation.", "rejected": "IoT is the study of things found on the Internet."},
    {"prompt": "What is robotics?", "chosen": "Robotics is the branch of technology dealing with the design and construction of robots.", "rejected": "Robotics is a sport played by robots."},
    {"prompt": "Define virtual reality.", "chosen": "Virtual reality is a simulated environment created using computer technology.", "rejected": "Virtual reality is dreaming while awake."},
    {"prompt": "What is 5G technology?", "chosen": "5G is the fifth generation of wireless network technology for faster communication.", "rejected": "5G is five gigabytes of mobile data."},
    {"prompt": "Explain data mining.", "chosen": "Data mining is the process of discovering patterns in large data sets.", "rejected": "Data mining involves digging for data in computers."},
    {"prompt": "What is artificial neural network training?", "chosen": "It involves adjusting weights to minimize prediction errors.", "rejected": "It’s about teaching robots how to knit."},
    {"prompt": "What is DNA?", "chosen": "DNA is a molecule carrying genetic instructions in living organisms.", "rejected": "DNA is a brand of Wi-Fi router."},
    {"prompt": "Who was Nikola Tesla?", "chosen": "Nikola Tesla was an inventor and electrical engineer known for AC current development.", "rejected": "Nikola Tesla invented the smartphone."},
    {"prompt": "Define gravity.", "chosen": "Gravity is the natural force pulling objects toward each other.", "rejected": "Gravity is the glue of the Earth."},
    {"prompt": "What is the speed of light?", "chosen": "The speed of light is approximately 299,792 kilometers per second.", "rejected": "The speed of light depends on the color of the light."},
    {"prompt": "Explain time zones.", "chosen": "Time zones are regions where the same standard time is used.", "rejected": "Time zones are imaginary lines drawn by astronauts."},
    {"prompt": "What is photosynthesis used for?", "chosen": "Plants use photosynthesis to make food using sunlight.", "rejected": "Animals use photosynthesis to see in the dark."},
    {"prompt": "What is an atom?", "chosen": "An atom is the smallest unit of matter retaining the properties of an element.", "rejected": "An atom is a small battery."},
    {"prompt": "Who was Marie Curie?", "chosen": "Marie Curie was a physicist and chemist known for her research on radioactivity.", "rejected": "Marie Curie was a singer."},
    {"prompt": "Define global warming.", "chosen": "Global warming is the long-term rise in Earth’s average temperature due to greenhouse gases.", "rejected": "Global warming happens when the sun gets angry."},
    {"prompt": "What is the capital of France?", "chosen": "The capital of France is Paris.", "rejected": "The capital of France is London."},
    {"prompt": "Who painted the Mona Lisa?", "chosen": "The Mona Lisa was painted by Leonardo da Vinci.", "rejected": "The Mona Lisa was painted by Picasso."},
    {"prompt": "What is the main language of Brazil?", "chosen": "The main language of Brazil is Portuguese.", "rejected": "The main language of Brazil is Spanish."},
    {"prompt": "What is the boiling point of water?", "chosen": "The boiling point of water is 100°C at standard pressure.", "rejected": "Water boils when it feels angry."},
]


In [3]:
# Load Tokenizer and Model
base_model_name = "EleutherAI/gpt-neo-125M"
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(base_model_name)
model.resize_token_embeddings(len(tokenizer))
model.train()



tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/526M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

GPTNeoForCausalLM(
  (transformer): GPTNeoModel(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPTNeoBlock(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPTNeoAttention(
          (attention): GPTNeoSelfAttention(
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (resid_dropout): Dropout(p=0.0, inplace=False)
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=False)
            (q_proj): Linear(in_features=768, out_features=768, bias=False)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTNeoMLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (c_proj): Linear(in_fe

In [4]:
# Format data for SFT
sft_data = [
    {"input_ids": tokenizer(d["prompt"] + d["chosen"], truncation=True,
                            padding="max_length", max_length=64,
                            return_tensors="pt")["input_ids"].squeeze(),
                            "labels": tokenizer(d["prompt"] + d["chosen"],
                                                truncation=True,
                                                padding="max_length",
                                                max_length=64,
                                                return_tensors="pt")["input_ids"].squeeze()}
    for d in data
]

In [5]:
class SFTDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data
    def __len__(self): return len(self.data)
    def __getitem__(self, idx): return self.data[idx]

sft_dataset = SFTDataset(sft_data)

In [7]:
training_args = TrainingArguments(
    output_dir="./sft_model",
    per_device_train_batch_size=2,
    num_train_epochs=10,
    logging_steps=1,
    save_strategy="no",
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=sft_dataset,
    tokenizer=tokenizer
)

trainer.train()

/tmp/ipython-input-4208811879.py:10: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
1,0.276400
2,0.266400
3,0.500000
4,0.285700
5,0.307700
6,0.320900
7,0.346000
8,0.326500
9,0.202900
10,0.346100


TrainOutput(global_step=260, training_loss=0.10500391920168813, metrics={'train_runtime': 19.3835, 'train_samples_per_second': 26.311, 'train_steps_per_second': 13.413, 'total_flos': 16651952455680.0, 'train_loss': 0.10500391920168813, 'epoch': 10.0})

In [8]:
model.save_pretrained("./sft_finetuned_model")
tokenizer.save_pretrained("./sft_finetuned_model")

('./sft_finetuned_model/tokenizer_config.json',
 './sft_finetuned_model/special_tokens_map.json',
 './sft_finetuned_model/vocab.json',
 './sft_finetuned_model/merges.txt',
 './sft_finetuned_model/added_tokens.json',
 './sft_finetuned_model/tokenizer.json')

## Reward Modeling

In [9]:
from transformers import AutoModelForSequenceClassification

# RM is a classifier scoring chosen > rejected
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
tokenizer.pad_token = tokenizer.eos_token
reward_model = AutoModelForSequenceClassification.from_pretrained(base_model_name, num_labels=1)
reward_model.resize_token_embeddings(len(tokenizer))
reward_model.config.pad_token_id = tokenizer.pad_token_id

reward_model.train()

Some weights of GPTNeoForSequenceClassification were not initialized from the model checkpoint at EleutherAI/gpt-neo-125M and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPTNeoForSequenceClassification(
  (transformer): GPTNeoModel(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPTNeoBlock(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPTNeoAttention(
          (attention): GPTNeoSelfAttention(
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (resid_dropout): Dropout(p=0.0, inplace=False)
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=False)
            (q_proj): Linear(in_features=768, out_features=768, bias=False)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTNeoMLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (c_proj)

In [10]:
# Prepare RM dataset
rm_data = []
for ex in data:
    rm_data.append({"text": ex["prompt"] + ex["chosen"], "label": 1.0})
    rm_data.append({"text": ex["prompt"] + ex["rejected"], "label": 0.0})

class RMDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data
    def __len__(self): return len(self.data)
    def __getitem__(self, idx):
        enc = tokenizer(self.data[idx]["text"], truncation=True, padding="max_length", max_length=64, return_tensors="pt")
        return {
            "input_ids": enc["input_ids"].squeeze(),
            "attention_mask": enc["attention_mask"].squeeze(),
            "labels": torch.tensor(self.data[idx]["label"])
        }

rm_dataset = RMDataset(rm_data)

In [12]:
reward_args = TrainingArguments(
    output_dir="./rm_model",
    per_device_train_batch_size=2,
    num_train_epochs=10,
    logging_steps=1,
    save_strategy="no",
    report_to="none",
)

reward_trainer = Trainer(
    model=reward_model,
    args=reward_args,
    train_dataset=rm_dataset,
    tokenizer=tokenizer
)

reward_trainer.train()

/tmp/ipython-input-2555860416.py:10: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  reward_trainer = Trainer(


Step,Training Loss
1,0.034000
2,0.572700
3,0.657100
4,0.121200
5,0.096800
6,0.085300
7,0.001700
8,0.227400
9,0.515300
10,0.398700


TrainOutput(global_step=510, training_loss=0.06918731503495414, metrics={'train_runtime': 37.9173, 'train_samples_per_second': 26.901, 'train_steps_per_second': 13.45, 'total_flos': 33304205721600.0, 'train_loss': 0.06918731503495414, 'epoch': 10.0})

In [13]:
reward_model.save_pretrained("./reward_model")
#tokenizer.save_pretrained("./sft_finetuned_model")

## Direct Preference Optimization (DPO)

In [14]:
from torch.optim import AdamW
import torch.nn.functional as F
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForCausalLM.from_pretrained("./sft_finetuned_model")
model.resize_token_embeddings(len(tokenizer))
model.train()
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-6)

In [15]:
# DPO loss from earlier
def dpo_loss(logits_chosen, logits_rejected, beta=0.1):
    logp_chosen = torch.log_softmax(logits_chosen[:, -1, :], dim=-1)
    logp_rejected = torch.log_softmax(logits_rejected[:, -1, :], dim=-1)
    chosen_token_id = logits_chosen[:, -1, :].argmax(dim=-1)
    rejected_token_id = logits_rejected[:, -1, :].argmax(dim=-1)
    chosen_scores = logp_chosen.gather(1, chosen_token_id.unsqueeze(1))
    rejected_scores = logp_rejected.gather(1, rejected_token_id.unsqueeze(1))
    diff = chosen_scores - rejected_scores
    return -torch.log(torch.sigmoid(beta * diff)).mean()

In [17]:
# DPO Training loop
epochs = 10
for epoch in range(epochs):
    for example in data:
        chosen_enc = tokenizer(example["prompt"] + example["chosen"], return_tensors="pt", truncation=True, padding="max_length", max_length=64)
        rejected_enc = tokenizer(example["prompt"] + example["rejected"], return_tensors="pt", truncation=True, padding="max_length", max_length=64)
        chosen_enc = {k: v.to(device) for k, v in chosen_enc.items()}
        rejected_enc = {k: v.to(device) for k, v in rejected_enc.items()}
        out_chosen = model(**chosen_enc)
        out_rejected = model(**rejected_enc)
        loss = dpo_loss(out_chosen.logits, out_rejected.logits)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(f"[DPO] Epoch {epoch} | Loss: {loss.item():.4f}")

[DPO] Epoch 0 | Loss: 0.6576
[DPO] Epoch 0 | Loss: 0.6204
[DPO] Epoch 0 | Loss: 0.6581
[DPO] Epoch 0 | Loss: 0.5800
[DPO] Epoch 0 | Loss: 0.6170
[DPO] Epoch 0 | Loss: 0.6206
[DPO] Epoch 0 | Loss: 0.6412
[DPO] Epoch 0 | Loss: 0.6699
[DPO] Epoch 0 | Loss: 0.6922
[DPO] Epoch 0 | Loss: 0.6529
[DPO] Epoch 0 | Loss: 0.6475
[DPO] Epoch 0 | Loss: 0.6949
[DPO] Epoch 0 | Loss: 0.6532
[DPO] Epoch 0 | Loss: 0.6928
[DPO] Epoch 0 | Loss: 0.6467
[DPO] Epoch 0 | Loss: 0.6737
[DPO] Epoch 0 | Loss: 0.6696
[DPO] Epoch 0 | Loss: 0.5717
[DPO] Epoch 0 | Loss: 0.6916
[DPO] Epoch 0 | Loss: 0.6215
[DPO] Epoch 0 | Loss: 0.6367
[DPO] Epoch 0 | Loss: 0.6279
[DPO] Epoch 0 | Loss: 0.7375
[DPO] Epoch 0 | Loss: 0.5423
[DPO] Epoch 0 | Loss: 0.6745
[DPO] Epoch 0 | Loss: 0.6762
[DPO] Epoch 0 | Loss: 0.7306
[DPO] Epoch 0 | Loss: 0.6997
[DPO] Epoch 0 | Loss: 0.6288
[DPO] Epoch 0 | Loss: 0.6355
[DPO] Epoch 0 | Loss: 0.6810
[DPO] Epoch 0 | Loss: 0.7333
[DPO] Epoch 0 | Loss: 0.6588
[DPO] Epoch 0 | Loss: 0.6177
[DPO] Epoch 0 

In [18]:
model.save_pretrained("./fine_tuned_dpo_model")
tokenizer.save_pretrained("./fine_tuned_dpo_model")

('./fine_tuned_dpo_model/tokenizer_config.json',
 './fine_tuned_dpo_model/special_tokens_map.json',
 './fine_tuned_dpo_model/vocab.json',
 './fine_tuned_dpo_model/merges.txt',
 './fine_tuned_dpo_model/added_tokens.json',
 './fine_tuned_dpo_model/tokenizer.json')